# SGTI Scoring Comparison



**Report (3-5 Pages)**
- Problems encountered
- Hit rate percentages

**Work done**
- Response matching 
- Score matching
- Visualising outputs

**Load packages and data**
-

In [345]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## For Response ============================

resp_ext = pd.read_excel(r'SGTI 2021 - Extraction.xls', sheet_name = 'Responses', index_col= 'Unnamed: 1', header = 2)
resp_sgti = pd.read_excel(r'SGTI 2021 - Extraction.xls', sheet_name = 'SGTI Coding All', index_col= 'COMPANY NAME', header = 0)

## For Scoring =============================

scor_ext = pd.read_excel(r'SGTI 2021 - Extraction.xls', sheet_name = 'Scores', index_col= 'Unnamed: 1', header = 2)
scor_sgti = pd.read_excel(r'SGTI 2021 - Extraction.xls', sheet_name = 'SGTI Scoring All', index_col= 'COMPANY NAME', header = 0)

**Pre-processing data**
-

In [346]:
# Clean extracted dfs
def clean_ext(ext_df): 
    ext_df  = ext_df.iloc[1:, 3:]
    return ext_df

# Cleaning sgti dfs
def clean_sgti(sgti_df):
    sgti_df = sgti_df.iloc[:, 3:]
    sgti_df = sgti_df.dropna(how= 'all', axis = 1).infer_objects()
    return sgti_df

In [347]:
# Assign df and verify shape
ext_list = resp_ext, scor_ext = clean_ext(resp_ext), clean_ext(scor_ext)
print(['Shape is ' + str(i.shape) for i in ext_list])

['Shape is (57, 173)', 'Shape is (57, 173)']


In [348]:
# Assign df and verify shape
sgti_list = resp_sgti, scor_sgti = clean_sgti(resp_sgti), clean_sgti(scor_sgti)
print(['Shape is ' + str(i.shape) for i in sgti_list])

['Shape is (130, 165)', 'Shape is (519, 104)']


**Creating a standard format**
-
**For extraction**
- Obtain list of companies from extraction
- Filter down all matching questions, count all missing and extra columns and report

**For original**
- Obtain list of Questions for 2021 testing
- Filter down to just the companies from extraction

**Output**
- A dataframe with all SGTI provided columns, but extracted data

In [349]:
def convert_dtypes(fuk): 
    return fuk.set_axis([str(i) for i in fuk.columns.tolist()], axis =1)

dfs = resp_ext, resp_sgti, scor_ext, scor_ext
resp_ext, resp_sgti, scor_ext, scor_ext = [convert_dtypes(fuk) for fuk in dfs]

In [350]:
def parse_dfs(ext_df, sgti_df):

    # In-scope questions list
    questions = [str(i) for i in sgti_df.columns.tolist()]

    # Matching the columns against qn list
    output, cols = pd.DataFrame(), [str(i) for i in ext_df.columns.tolist()]

    # ext_df = ext_df.set_axis(cols, axis = 1)
    # sgti_df = sgti_df.set_axis(questions, axis = 1)
    missing_qns = []
    for i in questions:
        if i in cols:
            output[i] = ext_df[i].values.tolist()
        else:
            missing_qns.append(i)

    # Print metrics (Uncomment to see)
    # print('Number of missing questions: ' + str(len(missing_qns)))
    # print('Missing questions are: ' + str(missing_qns))
    # print('Data has ' + str(output.shape[0])+' rows and ' +str(output.shape[1])+' columns')


    return output.set_axis(ext_df.index, axis =0 , inplace = False).replace(['Yes', 'No'], ['Y', 'N']).fillna(0)

**Responses**
-

In [351]:
resp = parse_dfs(resp_ext, resp_sgti)

**Scoring**
-

In [352]:
## Matching scoring
scor = parse_dfs(scor_ext, scor_sgti)

**Matching Algo**
-

- Compare the differences between non-missing questions 
- Show percentages and scores for each question


**Demo**
- Performed using one company 3CNERGY LIMITED

In [353]:
# Standardising the original df, needs generated ext cols to create a 1-1 match
def compare_dfs(ext_df, sgti_df, company):

    # Obtain comparables using previous function
    previous = parse_dfs(ext_df, sgti_df)
    companies = list(ext_df.index.values)
    final_cols = previous.columns.tolist()
    final_col_len = len(final_cols)

    sgti = sgti_df.loc[companies, final_cols].fillna(0)

    # Comparing the two 
    comp = sgti.compare(previous, align_axis = 0)
    comp = comp.loc[company].dropna(how = 'all', axis =1 )
    comp_len = len(comp)

    accuracy = round(100*(1-(comp_len/final_col_len)), 1)

    # print('Accuracy: ' + str(accuracy))
    return comp, accuracy

In [354]:
compare_dfs(resp_ext, resp_sgti, '3CNERGY LIMITED')[0]

,28,29,32,37,C.1.12,D.6.3,D.8.1,NCG13,67,72,73,75,76,NCG10,79,RGG02,21,71,B.1.1(B),C.1.1(B),RGG03,85
self,3,2,B3,E,N,Y,Y,N,3,0,0,0,0,3,0.0,0,2,2.0,0,0,0,2.0
other,3,2,Disclosure in Bands,N,0.0,0.0,0.0,Y,Y,N,N,N,N,Y,N,N,Y,Y,N,N,N,0.0


In [355]:
compare_dfs(scor_ext, scor_sgti, '3CNERGY LIMITED')[0]

,3,10,11,12-13,37,A.3.16,A.3.15,B.5.2,D.6.4,NCG13,67,NCG10,21,71
self,3.0,2.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,3.0,3.0,2.0,2.0
other,0.0,1.0,10.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


**Average Accuracy metric**
-

- Obtains the average of all accuracy scores for all companies that are extracted.



In [360]:
def obtain_avg_acc(ext_df, sgti_df):
    companies = list(ext_df.index.values)   
    list_acc = [compare_dfs(ext_df, sgti_df, i)[1] for i in companies]
    return np.mean(list_acc)

In [359]:
obtain_avg_acc(resp_ext, resp_sgti)

98.7

In [358]:
obtain_avg_acc(scor_ext, scor_sgti)

97.8